In [ ]:
#! pip install highlight_text

In [ ]:
import requests
import time
from bs4 import BeautifulSoup
import json
from tqdm import tqdm
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from highlight_text import fig_text
from matplotlib.patches import Arc
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import re

In [ ]:
# Entering the league's  link
link = "https://understat.com/league/Ligue_1/2024"
res = requests.get(link)
soup = BeautifulSoup(res.content,'lxml')
scripts = soup.find_all('script')
# Get the players stats 
strings = scripts[3].string 
# Getting rid of unnecessary characters from json data
ind_start = strings.index("('")+2 
ind_end = strings.index("')") 
json_data = strings[ind_start:ind_end]
json_data = json_data.encode('utf8').decode('unicode_escape')
data = json.loads(json_data)
# Creating the dataframe
all_shots = pd.DataFrame(data)
# Changing the data type using pd.to_numeric() function
all_shots["id"] = pd.to_numeric(all_shots["id"])
idd = all_shots["id"].values.tolist()

In [ ]:
shoots_list = []  # Utiliser une liste pour stocker les DataFrames temporairement

for i in tqdm(idd):
    try:
        url = f"https://understat.com/player/{i}"
        r = requests.get(url)
        r.raise_for_status()  # Vérifie si la requête a réussi
        soup = BeautifulSoup(r.content, 'lxml')
        scripts = soup.find_all('script')
        strings = scripts[3].string
        ind_start = strings.index("('") + 2
        ind_end = strings.index("')")
        json_data = strings[ind_start:ind_end]
        json_data = json_data.encode('utf8').decode('unicode_escape')
        data = json.loads(json_data)
        shoots_list.append(pd.DataFrame(data))  # Ajouter le DataFrame à la liste
    except Exception as e:
        print(f"Erreur pour le joueur {i}: {e}")
    finally:
        time.sleep(1)  # Ajouter un délai pour éviter de surcharger le serveur

# Concaténer tous les DataFrames en une seule fois
shoots = pd.concat(shoots_list, ignore_index=True)

In [ ]:
brest_shoots = shoots[(shoots["h_team"]=="Brest")|((shoots["a_team"]=="Brest"))]
print(brest_shoots.shape)
display(brest_shoots.head(9))

In [ ]:
# Changing data types
brest_shoots['X'] = brest_shoots['X'].astype('float64')
brest_shoots['Y'] = brest_shoots['Y'].astype('float64')

# Adjustind dimensions for soccer pitch
brest_shoots['X1'] = (brest_shoots['X']/100)*105*100
brest_shoots['Y1'] = (brest_shoots['Y']/100)*68*100

In [ ]:
brest_shoots.to_csv("brest_shoots.csv", index=False)

# Football Pitch 

In [ ]:
def football_pitch(x_min=0, x_max=105,
               y_min=0, y_max=68,
               pitch_color="#f0f0f0",
               line_color='black',
               line_thickness=1.5,
               point_size=20,
               orientation="horizontal",
               aspect="full",
               axis='off',
               ax=None
               ):

    if not ax:
        raise TypeError("This function is intended to be used with an existing fig and ax in order to allow flexibility in plotting of various sizes and in subplots.")


    if orientation.lower().startswith("h"):
        first = 0
        second = 1
        arc_angle = 0

        if aspect == "half":
            ax.set_xlim(x_max / 2, x_max + 5)

    elif orientation.lower().startswith("v"):
        first = 1
        second = 0
        arc_angle = 90

        if aspect == "half":
            ax.set_ylim(x_max / 2, x_max + 5)

    
    else:
        raise NameError("You must choose one of horizontal or vertical")
    
    ax.axis(axis)


    x_conversion = x_max / 100
    y_conversion = y_max / 100

    pitch_x = [0,5.8,11.5,17,50,83,88.5,94.2,100] # x dimension markings
    pitch_x = [x * x_conversion for x in pitch_x]

    pitch_y = [0, 21.1, 36.6, 50, 63.2, 78.9, 100] # y dimension markings
    pitch_y = [x * y_conversion for x in pitch_y]

    goal_y = [45.2, 54.8] # goal posts
    goal_y = [x * y_conversion for x in goal_y]

    # side and goal lines
    lx1 = [x_min, x_max, x_max, x_min, x_min]
    ly1 = [y_min, y_min, y_max, y_max, y_min]

    # outer box
    lx2 = [x_max, pitch_x[5], pitch_x[5], x_max]
    ly2 = [pitch_y[1], pitch_y[1], pitch_y[5], pitch_y[5]]

    lx3 = [0, pitch_x[3], pitch_x[3], 0]
    ly3 = [pitch_y[1], pitch_y[1], pitch_y[5], pitch_y[5]]

    # goals
    lx4 = [x_max, x_max+2, x_max+2, x_max]
    ly4 = [goal_y[0], goal_y[0], goal_y[1], goal_y[1]]

    lx5 = [0, -2, -2, 0]
    ly5 = [goal_y[0], goal_y[0], goal_y[1], goal_y[1]]

    # 6 yard box
    lx6 = [x_max, pitch_x[7], pitch_x[7], x_max]
    ly6 = [pitch_y[2],pitch_y[2], pitch_y[4], pitch_y[4]]

    lx7 = [0, pitch_x[1], pitch_x[1], 0]
    ly7 = [pitch_y[2],pitch_y[2], pitch_y[4], pitch_y[4]]


    # Halfline, penalty spots, and kickoff spot
    lx8 = [pitch_x[4], pitch_x[4]]
    ly8 = [0, y_max]

    lines = [
        [lx1, ly1],
        [lx2, ly2],
        [lx3, ly3],
        [lx4, ly4],
        [lx5, ly5],
        [lx6, ly6],
        [lx7, ly7],
        [lx8, ly8],
        ]

    points = [
        [pitch_x[6], pitch_y[3]],
        [pitch_x[2], pitch_y[3]],
        [pitch_x[4], pitch_y[3]]
        ]

    circle_points = [pitch_x[4], pitch_y[3]]
    arc_points1 = [pitch_x[6], pitch_y[3]]
    arc_points2 = [pitch_x[2], pitch_y[3]]


    for line in lines:
        ax.plot(line[first], line[second],
                color=line_color,
                lw=line_thickness,
                zorder=-1)

    for point in points:
        ax.scatter(point[first], point[second],
                   color=line_color,
                   s=point_size,
                   zorder=-1)

    circle = plt.Circle((circle_points[first], circle_points[second]),
                        x_max * 0.088,
                        lw=line_thickness,
                        color=line_color,
                        fill=False,
                        zorder=-1)

    ax.add_artist(circle)

    arc1 = Arc((arc_points1[first], arc_points1[second]),
               height=x_max * 0.088 * 2,
               width=x_max * 0.088 * 2,
               angle=arc_angle,
               theta1=128.75,
               theta2=231.25,
               color=line_color,
               lw=line_thickness,
               zorder=-1)

    ax.add_artist(arc1)

    arc2 = Arc((arc_points2[first], arc_points2[second]),
               height=x_max * 0.088 * 2,
               width=x_max * 0.088 * 2,
               angle=arc_angle,
               theta1=308.75,
               theta2=51.25,
               color=line_color,
               lw=line_thickness,
               zorder=-1)

    ax.add_artist(arc2)

    ax.set_aspect("equal")

    return ax

In [ ]:
open_play = brest_shoots[brest_shoots["situation"]=="OpenPlay"]
free_kick = brest_shoots[brest_shoots["situation"]=="DirectFreekick"]
corner =  brest_shoots[brest_shoots["situation"]=="FromCorner"]
set_piece = brest_shoots[brest_shoots["situation"]=="SetPiece"]

In [ ]:
from matplotlib import cm  # Importer pour utiliser la palette Spectral

fig = plt.figure(figsize=(20, 30), constrained_layout=True)
gs = fig.add_gridspec(nrows=1, ncols=2)

# Palette Spectral inversée
spectral_palette = cm.get_cmap('Spectral_r')  # Ajouter "_r" pour inverser la palette

ax = fig.add_subplot(gs[0])
football_pitch(orientation="vertical", aspect="half", line_color="black", ax=ax)
sns.kdeplot(x=free_kick["Y1"], y=free_kick["X1"], fill=True, cmap=spectral_palette, levels=100)

ax1 = fig.add_subplot(gs[1])
football_pitch(orientation="vertical", aspect="half", line_color="black", ax=ax1)
sns.kdeplot(x=corner["Y1"], y=corner["X1"], fill=True, cmap=spectral_palette, levels=100)

# Indicateurs

### Nombre de buts

In [ ]:
# Filtrer les tirs où le joueur est "Ousmane Dembélé" et le résultat est "Goal"
Ludovic_Ajorque_goals = brest_shoots[(brest_shoots['player'] == "Ludovic Ajorque") & (brest_shoots['result'] == "Goal")]

# Calculer le nombre total de buts
total_goals = len(Ludovic_Ajorque_goals)

print(f"Nombre total de buts marqués par Ludovic Ajorque : {total_goals}")

In [ ]:
# Filtrer les tirs où le joueur est "Ousmane Dembélé" et le résultat est "Goal"
Ludovic_Ajorque_shoot = brest_shoots[(brest_shoots['player'] == "Ludovic Ajorque") & ((brest_shoots['result'] == "MissedShots") | (brest_shoots['result'] == "ShotOnPost"))]
# Calculer le nombre total de buts
total_shoot = len(Ludovic_Ajorque_shoot)

print(f"Nombre total de buts marqués par Ludovic Ajorque : {total_shoot}")

### Nombre d'apparitions

In [ ]:
# Filtrer les lignes où le joueur est "Ludovic Ajorque"
ajorque_presence = brest_shoots[brest_shoots['player'] == "Ludovic Ajorque"]

# Compter le nombre de fois où il est présent
presence_count = ajorque_presence['match_id'].nunique()  # Utiliser 'match_id' pour éviter les doublons

print(f"Ludovic Ajorque était présent dans {presence_count} matchs de Brest.")

### value market joueur

In [ ]:
# Lecture du fichier players (sans "date")
players = pd.read_csv("data/players.csv", usecols=[
    "player_id", "name", "position", "sub_position", "foot",
    "height_in_cm", "country_of_citizenship", "current_club_name","player_id"
])

# Lecture des valuations (avec "date")
valuations = pd.read_csv("data/player_valuations.csv")

# Jointure
df = pd.merge(players, valuations, on="player_id")


In [ ]:
df=df[df['name'] == "Kylian Mbappé"]
df

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import PchipInterpolator
from matplotlib.patches import PathPatch
from matplotlib.path import Path
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import matplotlib.image as mpimg

# Données
annees = [2017, 2022, 2024, 2025]
valeurs = [0.5, 16, 3, 4]

# Lissage sans dépassement
annees_np = np.array(annees)
valeurs_np = np.array(valeurs)
x_liss = np.linspace(annees_np.min(), annees_np.max(), 300)
spl = PchipInterpolator(annees_np, valeurs_np)
y_liss = spl(x_liss)

# Couleurs selon les valeurs
norm = plt.Normalize(min(valeurs), max(valeurs))
cmap = plt.get_cmap('RdYlGn')

# Dictionnaire d'images associées à chaque année
logos = {
    2017: "images/clermont.svg.png",
    2022: "images/strasbourg.svg.png",
    2024: "images/mainz.png",
    2025: "images/sb29.svg.png"
}

# Zoom personnalisé par image
zoom_par_image = {
    2017: 0.05,
    2022: 0.05,
    2024: 0.3,
    2025: 0.04
}

# Création du graphe HD sans fond coloré
fig, ax = plt.subplots(figsize=(14, 6), dpi=300)
ax.set_facecolor('none')
fig.patch.set_facecolor('none')

# Tracé de la courbe avec dégradé
for i in range(len(x_liss) - 1):
    color = cmap(norm(y_liss[i]))
    ax.plot([x_liss[i], x_liss[i + 1]], [y_liss[i], y_liss[i + 1]], color=color, lw=2)

# Affichage des points, textes et images
for a, v in zip(annees, valeurs):
    ax.plot(a, v, 'o', color='crimson')

    # Texte des valeurs en dessous de la courbe maintenant
    ax.text(a, v - 1.2, f"{v}M€", ha='center', va='top',
            fontsize=14, fontweight='bold', color='black')

    # Texte des années
    ax.text(a, -1.2, f"{a}", ha='center', va='top',
            fontsize=13, fontweight='bold', color='black')

    # Ajout du logo correspondant avec zoom personnalisé
    if a in logos:
        img = mpimg.imread(logos[a])
        zoom_val = zoom_par_image.get(a, 0.08)
        imagebox = OffsetImage(img, zoom=zoom_val)
        ab = AnnotationBbox(imagebox, (a, v + 2.2), frameon=False)
        ax.add_artist(ab)

# Effet néon
def neon_effect(ax, x, y, linewidth=6, alpha=0.3, color='cyan'):
    path = Path([(x[i], y[i]) for i in range(len(x))], closed=False)
    patch = PathPatch(path, facecolor='none', lw=linewidth, edgecolor=color, alpha=alpha, linestyle='-', zorder=1)
    ax.add_patch(patch)

neon_effect(ax, x_liss, y_liss)

# Ajustements finaux
ax.set_ylim(min(y_liss) - 4, max(y_liss) + 6)
ax.axis('off')
plt.tight_layout()

# Export PNG HD avec fond transparent
plt.savefig("graph_valorisation_logos.png", dpi=300, bbox_inches='tight', transparent=True)
plt.show()


In [ ]:
plt.savefig("graph_hd.png", dpi=300, bbox_inches='tight', facecolor=fig.get_facecolor())


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import PchipInterpolator
from matplotlib.patches import PathPatch
from matplotlib.path import Path
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import matplotlib.image as mpimg

# Données
annees = [2017, 2022, 2024, 2025]
valeurs = [0.5, 16, 3, 4]

# Lissage sans dépassement
annees_np = np.array(annees)
valeurs_np = np.array(valeurs)
x_liss = np.linspace(annees_np.min(), annees_np.max(), 300)
spl = PchipInterpolator(annees_np, valeurs_np)
y_liss = spl(x_liss)

# Couleurs selon les valeurs
norm = plt.Normalize(min(valeurs), max(valeurs))
cmap = plt.get_cmap('RdYlGn')

# Dictionnaire d'images associées à chaque année
logos = {
    2017: "images/clermont.svg.png",
    2022: "images/strasbourg.svg.png",
    2024: "images/mainz.png",
    2025: "images/sb29.svg.png"
}

# Zoom personnalisé par image
zoom_par_image = {
    2017: 0.05,
    2022: 0.05,
    2024: 0.3,
    2025: 0.04
}

# Création du graphe HD sans fond coloré
fig, ax = plt.subplots(figsize=(14, 6), dpi=300)
ax.set_facecolor('none')
fig.patch.set_facecolor('none')

# Tracé de la courbe avec dégradé
for i in range(len(x_liss) - 1):
    color = cmap(norm(y_liss[i]))
    ax.plot([x_liss[i], x_liss[i + 1]], [y_liss[i], y_liss[i + 1]], color=color, lw=2)

# Affichage des points, textes et images
for a, v in zip(annees, valeurs):
    ax.plot(a, v, 'o', color='crimson')

    # Texte des valeurs en blanc
    ax.text(a, v - 0.8, f"{v}M€", ha='center', va='top',
            fontsize=14, fontweight='bold', color='white')

    # Texte des années en blanc
    ax.text(a, -1.2, f"{a}", ha='center', va='top',
            fontsize=13, fontweight='bold', color='white')

    # Ajout du logo correspondant avec zoom personnalisé
    if a in logos:
        img = mpimg.imread(logos[a])
        zoom_val = zoom_par_image.get(a, 0.08)
        imagebox = OffsetImage(img, zoom=zoom_val)
        ab = AnnotationBbox(imagebox, (a, v + 2.2), frameon=False)
        ax.add_artist(ab)

# Effet néon
def neon_effect(ax, x, y, linewidth=6, alpha=0.3, color='cyan'):
    path = Path([(x[i], y[i]) for i in range(len(x))], closed=False)
    patch = PathPatch(path, facecolor='none', lw=linewidth, edgecolor=color, alpha=alpha, linestyle='-', zorder=1)
    ax.add_patch(patch)

neon_effect(ax, x_liss, y_liss)

# Ajustements finaux
ax.set_ylim(min(y_liss) - 4, max(y_liss) + 6)
ax.axis('off')
plt.tight_layout()

# Export PNG HD avec fond transparent
plt.savefig("graph_valorisation_logos.png", dpi=300, bbox_inches='tight', transparent=True)
plt.show()
